# <p style="text-align: center; font-size: 50px; font-weight: bold; color: #4CAF50;">Vision FX</p>
### <p style="text-align: center; font-size: 24px; font-weight: bold; color: #2196F3;">An Image Filtering Application</p>
#### <p style="text-align: center; font-size: 20px; color: #FF5722;">By: Farhan Ahmad & Momena Azhar</p>

In [5]:
!pip install opencv-python

In [9]:
!pip install opencv-python-headless

In [6]:
!pip install tk

In [17]:
!pip install PyQt5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 21.4 MB/s eta 0:00:00


In [ ]:
import sys
import cv2
import numpy as np
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QPushButton, QFileDialog, QVBoxLayout, QHBoxLayout, QComboBox
from PyQt5.QtGui import QPixmap, QImage, QFont
from PyQt5.QtCore import Qt  # Import Qt for text alignment

class ImageFilterApp(QWidget):
    def __init__(self):
        """Initialize the GUI application."""
        super().__init__()
        self.initUI()
        self.img = None  # Stores the uploaded image
        self.processed_img = None  # Stores the filtered image

    def initUI(self):
        """Set up the GUI layout and components."""
        self.setWindowTitle("Image Filters GUI")
        self.setStyleSheet("background-color: black;")  # Set background color
        self.setGeometry(100, 100, 800, 600)  # Set window size and position

        layout = QVBoxLayout()

        # 🔹 Welcome Header
        self.header_label = QLabel("WELCOME TO VISION FX")
        self.header_label.setFont(QFont("Arial", 24, QFont.Bold))  # Set font size and bold text
        self.header_label.setStyleSheet("color: white; text-align: center;")  # Style the header
        self.header_label.setAlignment(Qt.AlignCenter)  # Align text to center
        layout.addWidget(self.header_label)

        # Upload button
        self.upload_btn = QPushButton("Upload Image")
        self.upload_btn.setStyleSheet("background-color: gray; color: white;")
        self.upload_btn.clicked.connect(self.upload_image)
        layout.addWidget(self.upload_btn)

        # "Select Filter" button
        self.select_filter_btn = QPushButton("Select Filter")
        self.select_filter_btn.setStyleSheet("background-color: gray; color: white;")
        self.select_filter_btn.setEnabled(False)  # Initially disabled until an image is uploaded
        self.select_filter_btn.clicked.connect(self.open_filter_dropdown)
        layout.addWidget(self.select_filter_btn)

        # Filter selection dropdown (hidden initially)
        self.filter_box = QComboBox()
        self.filter_box.addItems(["Grayscale", "Blurring", "Edge Detection", "HSV", "Sepia"])
        self.filter_box.setVisible(False)  # Hide initially
        self.filter_box.currentIndexChanged.connect(self.filter_selected)  # Apply filter on selection
        layout.addWidget(self.filter_box)

        # Image display layout
        image_layout = QHBoxLayout()

        # Label to display the original image
        self.original_label = QLabel()
        self.original_label.setStyleSheet("background-color: black;")
        image_layout.addWidget(self.original_label)

        # Label to display the processed image
        self.processed_label = QLabel()
        self.processed_label.setStyleSheet("background-color: black;")
        image_layout.addWidget(self.processed_label)

        layout.addLayout(image_layout)

        # Download button (Initially hidden)
        self.download_btn = QPushButton("Download Image")
        self.download_btn.setStyleSheet("background-color: green; color: white;")
        self.download_btn.setEnabled(False)  # Initially disabled until an image is processed
        self.download_btn.clicked.connect(self.download_image)
        layout.addWidget(self.download_btn)

        self.setLayout(layout)

    def upload_image(self):
        """Opens a file dialog to upload an image."""
        file_path, _ = QFileDialog.getOpenFileName(self, "Open Image File", "", "Images (*.png *.jpg *.jpeg *.bmp *.tiff)")
        if file_path:
            self.img = cv2.imread(file_path)  # Read the image using OpenCV
            self.display_image(self.img, self.original_label)  # Show the original image
            self.select_filter_btn.setEnabled(True)  # Enable filter selection button

    def open_filter_dropdown(self):
        """Opens the filter dropdown immediately when clicking 'Select Filter' button."""
        self.filter_box.setVisible(True)  # Show the dropdown
        self.filter_box.showPopup()  # Immediately open the dropdown

    def filter_selected(self):
        """Trigger the filter application when a filter is selected."""
        if self.filter_box.currentText():  
            self.apply_filter()

    def apply_filter(self):
        """Applies the selected filter to the uploaded image."""
        if self.img is not None:
            selected_filter = self.filter_box.currentText()
            self.processed_img = self.apply_selected_filter(self.img, selected_filter)
            self.display_image(self.processed_img, self.processed_label)
            self.download_btn.setEnabled(True)  # Enable download button

    def apply_selected_filter(self, img, filter_type):
        """Applies the specified filter to the image."""
        if filter_type == "Grayscale":
            return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        elif filter_type == "Blurring":
            return cv2.GaussianBlur(img, (15, 15), 0)  # Apply Gaussian blur
        elif filter_type == "Edge Detection":
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            return cv2.Canny(gray, 100, 200)  # Apply Canny edge detection
        elif filter_type == "HSV":
            return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)  # Convert to HSV color space
        elif filter_type == "Sepia":
            sepia_filter = np.array([[0.272, 0.534, 0.131],
                                     [0.349, 0.686, 0.168],
                                     [0.393, 0.769, 0.189]])  # Sepia tone matrix
            sepia_img = cv2.transform(img, sepia_filter)
            return np.clip(sepia_img, 0, 255).astype(np.uint8)  # Ensure valid pixel range
        return img  # Return the original image if no filter is applied

    def display_image(self, img, label):
        """Displays the image in the specified QLabel."""
        if len(img.shape) == 2:  # If grayscale image
            q_format = QImage.Format_Grayscale8
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for correct color display
            q_format = QImage.Format_RGB888

        height, width, ch = img.shape if len(img.shape) == 3 else (img.shape[0], img.shape[1], 1)
        bytes_per_line = ch * width
        q_img = QImage(img.data, width, height, bytes_per_line, q_format)
        pixmap = QPixmap.fromImage(q_img)
        pixmap = pixmap.scaled(300, 300)  # Standardized image size
        label.setPixmap(pixmap)
        label.setScaledContents(True)  # Ensure the image fits properly

    def download_image(self):
        """Saves the processed image to the user's computer."""
        if self.processed_img is not None:
            file_path, _ = QFileDialog.getSaveFileName(self, "Save Image", "", "JPEG Files (*.jpg);;PNG Files (*.png)")
            if file_path:
                cv2.imwrite(file_path, self.processed_img, [int(cv2.IMWRITE_JPEG_QUALITY), 95])  # Save with high quality

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ImageFilterApp()
    window.show()
    sys.exit(app.exec_())

2025-03-14 21:46:23.965 python[65468:5701585] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-14 21:46:23.965 python[65468:5701585] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2025-03-14 21:46:28.673 python[65468:5701585] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'
